In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": "<application-id>",
           "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="<scope-name>",key="<service-credential-key-name>"),
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<directory-id>/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://<file-system-name>@<storage-account-name>.dfs.core.windows.net/",
  mount_point = "/mnt/<mount-name>",
  extra_configs = configs)

In [ ]:
ratings_df_schema = "userId integer, movieId integer, rating float"
ratingsDF = spark.read.csv("/mnt/<path>/ratings.csv", header=True, schema=ratings_df_schema).cache()
ratingsDF.head(5)

In [ ]:
(trainingDF, testDF) = ratingsDF.randomSplit([0.8, 0.2], seed=21)

In [ ]:
from pyspark.ml.recommendation import ALS
als = (ALS()
       .setUserCol("userId")
       .setItemCol("movieId")
       .setRatingCol("rating")
       .setPredictionCol("predictions")
       .setMaxIter(3)
       .setSeed(21)
       .setRegParam(0.1)
       .setColdStartStrategy("drop")
       .setRank(12))   
alsModel = als.fit(trainingDF)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

regEval = RegressionEvaluator(predictionCol="predictions", labelCol="rating", metricName="mse")
predictedTestDF = alsModel.transform(testDF)
testMse = regEval.evaluate(predictedTestDF)

print('MSE on the test set is {0}'.format(testMse))

In [ ]:
alsModel.save("/mnt/<path>")

In [ ]:
dbutils.fs.unmount("/mnt/<path>")